In [1]:
import math

import numpy
import pandas

import ipywidgets as widgets
from IPython.display import display, clear_output

import gensim
from gensim.corpora import Dictionary
from gensim.models import TfidfModel

import pythainlp

from sklearn.metrics.pairwise import cosine_similarity

from data_tokenizer import load_corpus

from model.upgrade_sdc import UpgradeSDC
from model.sdc import SDC

### Load Data

In [2]:
file_name = 'ผู้บริโภค - TescoLotus.txt'

corpus, labels = load_corpus('../data/facebook/' + file_name)

len_corpus = len(corpus)
print('Total documents', len_corpus)

clusters = list(set(labels))
print(len(clusters), 'clusters')

f = open('../data/facebook/tokenized/tokenized_' + file_name)
tokenized_corpus = eval(f.read())
f.close()

Total documents 268
1 clusters


### Preprocess Corpus

#### Remove Words

In [3]:
dictionary = Dictionary(tokenized_corpus)
print('origin:', len(dictionary), 'words')

dictionary.filter_extremes(no_below=2, no_above=0.7, keep_n=len(dictionary))
print('filter frequent words:', len(dictionary), 'words')

letter_words = [id for id in range(len(dictionary)) if len(dictionary[id]) <= 1] 
dictionary.filter_tokens(bad_ids=letter_words)
print('filter letter words:', len(dictionary), 'words')

stopwords = pythainlp.corpus.stopwords.words('thai')
stopwords.append('นี้')
dictionary.add_documents([stopwords])
stopwords = [dictionary.token2id[word] for word in stopwords]
dictionary.filter_tokens(bad_ids=stopwords)
print('filter stop words:', len(dictionary), 'words')

origin: 1449 words
filter frequent words: 605 words
filter letter words: 604 words
filter stop words: 403 words


In [4]:
idx_corpus = [dictionary.doc2idx(doc) for doc in tokenized_corpus]

temp_corpus = []
for doc in idx_corpus:
    temp_corpus.append([dictionary[id] for id in doc if id >= 0])
idx_corpus = temp_corpus

#### Dimension Reduction

In [5]:
average_doc_size = 0
for doc in idx_corpus:
    average_doc_size += len(doc)
average_doc_size /= len(idx_corpus)
average_doc_size = math.ceil(average_doc_size)

df = dictionary.dfs
filtered_corpus = []
for doc in idx_corpus:
    new_doc = [(word, df[dictionary.token2id[word]]) for word in doc]
    new_doc.sort(reverse=True, key=lambda x: x[1])
    new_doc = new_doc[:average_doc_size]
    filtered_corpus.append([word for word, df in new_doc])

### Clustering

In [6]:
def get_onehot(corpus):
    new_dict = Dictionary(corpus)
#     new_dict.filter_extremes(no_below=2, no_above=1, keep_n=len(new_dict))

    unique_words = [new_dict[id] for id in range(len(new_dict))]
    array = numpy.zeros((len_corpus, len(unique_words)), dtype=float)    
    for i, doc in enumerate(corpus):
        for word in doc:
            array[i, new_dict.token2id[word]] += 1

        ## normalization
        if len(doc) != 0:
#             array[i] = numpy.divide(array[i], len(idx_corpus[i]))
            array[i] = numpy.divide(array[i], len(doc))
    
    return pandas.DataFrame(array, columns=unique_words, dtype=float)

In [7]:
def generate_result(predicted_labels):
    result = pandas.DataFrame()
    result['comment'] = corpus
    result['tokenized_comment'] = filtered_corpus
    result['label'] = labels
    result['predicted_label'] = predicted_labels
    return result

In [8]:
def eval_cluster(onehot_corpus, result):
    label_count = numpy.unique(result['predicted_label'])
    num_cluster = label_count[-1] + 1

    clusters = [[] for i in range(num_cluster)]
    corpus_centroid = []
    for i, label in result['predicted_label'].iteritems():
        clusters[label].append(numpy.array(onehot_corpus.iloc[i]))
        corpus_centroid.append(numpy.array(onehot_corpus.iloc[i]))
    corpus_centroid = numpy.mean(corpus_centroid, axis=0).reshape(1, -1)   

#     print('\tIntra cluster sim\tInter cluster sim\tIntra / Inter')
    compactness = 0
    centroids = []
    for i in range(num_cluster):
        size = len(clusters[i])
        if size != 0:
            centroid = numpy.mean(clusters[i], axis=0)
            centroids.append(centroid)
            centroid = centroid.reshape(1, -1)
            similarities = cosine_similarity(centroid, clusters[i])
            compactness += numpy.sum(similarities)

#             intra = numpy.sum(similarities) / size
#             inter = cosine_similarity(centroid, corpus_centroid)[0][0]
#             print(i, end='\t')
#             print(intra, end='\t')
#             print(inter, end='\t')
#             print(intra / inter)
    return compactness, centroids

In [15]:
min_samples = 7
eps = 0.32
epoch = 10

# onehot_corpus = get_onehot(idx_corpus)
onehot_corpus = get_onehot(filtered_corpus)

max_compactness = 0
for i in range(epoch):
    model = UpgradeSDC()
    _tpredicted_labels, _tmarks = model.predict(onehot_corpus, min_samples, eps)
    
    _tresult = generate_result(_tpredicted_labels)
    compactness, _tcentroids = eval_cluster(onehot_corpus, _tresult)
    
    if compactness > max_compactness:
        max_compactness = compactness
        predicted_labels = _tpredicted_labels
        marks = _tmarks
        result = _tresult
        centroids = _tcentroids
        
print(max_compactness)
label_count = numpy.unique(result['predicted_label'], return_counts=True) 
num_cluster = label_count[0][-1] + 1
print(label_count, '\n')

143.18919302853803
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15]), array([44, 16, 11, 30, 16, 18, 18,  9, 13,  8, 17, 15, 11, 15, 15, 12])) 



In [16]:
sims = cosine_similarity(centroids)
new_labels = [i for i in range(num_cluster)]
print(new_labels)
for i, row in reversed(list(enumerate(sims))):
    for j, value in reversed(list(enumerate(row[:i + 1]))):
        if i != j and value >= eps - eps / 20:
            print(i, j, value)
            new_labels = [new_labels[j] if label == new_labels[i] else label for label in new_labels]
print(new_labels)

grouped_labels = numpy.zeros(len_corpus)
for i, label in enumerate(predicted_labels):
    grouped_labels[i] = new_labels[label]
new_result = generate_result(grouped_labels)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
15 13 0.6096108378847768
15 11 0.43226598500062746
15 10 0.36761594848667045
15 8 0.48450067049442397
15 2 0.4514777087111918
15 1 0.6589595473778568
13 11 0.4616059184780303
13 8 0.40684081181430076
13 2 0.6879395650378226
13 1 0.5670507786353982
11 8 0.32352399582517527
11 5 0.3393126405305244
11 2 0.3260578179335435
11 1 0.586389800892499
10 1 0.38660770429536456
8 5 0.5677602361929177
8 2 0.3383174849913636
8 1 0.5499459045573912
6 5 0.313122169377218
6 1 0.3104758423106557
5 1 0.4307602225523216
4 1 0.3269203694086348
2 1 0.4673068712094792
[0, 5, 5, 3, 5, 5, 5, 7, 5, 9, 5, 5, 12, 5, 14, 5]


### Result

In [17]:
class Widget:
    def __init__(self, result, marks, column_name):
        self.result = result
        self.column_name = column_name
        self.marks = marks
        
        label_count = numpy.unique(result['predicted_label'])
        self.widget = widgets.ToggleButtons(
            options=[int(num) for num in label_count],
            disabled=False,
            button_style='',
        )
        
        self.widget.observe(self.on_click, names='index')
        self.on_click({'new' : 0})
        
    def on_click(self, change):
        clear_output()
        display(self.widget)
        new = self.widget.options[change['new']]
        for index, value in self.result[self.result['predicted_label'] == new][self.column_name].iteritems():
            if self.marks:
                if index in self.marks[0]:
                    print("@", end="")
                elif index in self.marks[1]:
                    print("*", end="")
            print(index, value)

In [18]:
w1 = Widget(new_result, marks, 'comment')

ToggleButtons(options=(0, 3, 5, 7, 9, 12, 14), value=0)

2 เลิกเปิดเพลง ข้าวแสนดี กับอีเครื่องกรองน้ำเพียว ได้แล้ว!!!!
8 อย่าบังคับน้องเข้าประชุมเชียร์ อย่าลงโทษโดยเหตุผลงี่เง่าๆ เกิดก่อนไม่กี่ปีเอง
14 เลือกผัก เลือกแล้ว เลือกอีก พอมาคิดเงินพนักงานยัดผักใส่ถุงจนผักหัก โอ้ยใจ ใจสลาย
29 โลตลาด พนง.ปากตลาด นินทาลค.เผาขน ตะโกนโหวกเหวกข้ามหัวลค. วันหวยออกสนั่นเป็นพิเศษ เห็นพฤติกรรมแล้วแย่ ปรับปรุงภาพลักษณ์เถอะ
35 Lotus สาขายโสธร-เปิดใหม่ พนง.แอบโกง Give Card ลูกค้า(กุเอง) จ่ายหลายใบแล้วทำเนียนว่ามีใบนึงชำรุดใช้ไม่ได้แล้วพอขอคืนก็สลับใบที่รูดใช้แล้วมาแทน แจ้งผู้จัดการขอดูกล้องวงจรปิดไม่ยอมให้ดูเลยต้องขู่ว่าจะแจ้งความถึงยอมคืนให้ แต่ไม่มีคำขอโทษใดๆ จาก พนง.-ติด…See more
43 สาขาพัทลุง เหม็นคาวมากกกกกกกกก แถมแผนกของกินของสดแม่งไปไหนหมด มีแต่เสื้อผ้าละ
56 ของมึงไม่ต้องถูกมากก็ได้ เน้นคุณภาพบ้าง ไส้กรอกเน่า แต่ยังไม่หมดอายุงี้อ่ะ (สาขาฟอร์จูน) ขนมปังขึ้นราแล้วยังวางขายอีก (สาขาลาดพร้าว) กีวี่หนอนเข้า กะหล่ำปลีเน่าใน ผักบุ้งเน่าแทรกในมัด (สาขาอ่อนนุช)
60 ของป้ายเหลืองพวกผัก เน่าจนไม่รู้จะเน่ายังไง คือทิ้งๆไปก็ได้มั่ง
68 ช่องจ่ายเยอะ เยอะเกินพนง.ใช่ไหม ไ

In [19]:
w2 = Widget(result, marks, 'comment')

ToggleButtons(options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15), value=0)

2 เลิกเปิดเพลง ข้าวแสนดี กับอีเครื่องกรองน้ำเพียว ได้แล้ว!!!!
8 อย่าบังคับน้องเข้าประชุมเชียร์ อย่าลงโทษโดยเหตุผลงี่เง่าๆ เกิดก่อนไม่กี่ปีเอง
14 เลือกผัก เลือกแล้ว เลือกอีก พอมาคิดเงินพนักงานยัดผักใส่ถุงจนผักหัก โอ้ยใจ ใจสลาย
29 โลตลาด พนง.ปากตลาด นินทาลค.เผาขน ตะโกนโหวกเหวกข้ามหัวลค. วันหวยออกสนั่นเป็นพิเศษ เห็นพฤติกรรมแล้วแย่ ปรับปรุงภาพลักษณ์เถอะ
35 Lotus สาขายโสธร-เปิดใหม่ พนง.แอบโกง Give Card ลูกค้า(กุเอง) จ่ายหลายใบแล้วทำเนียนว่ามีใบนึงชำรุดใช้ไม่ได้แล้วพอขอคืนก็สลับใบที่รูดใช้แล้วมาแทน แจ้งผู้จัดการขอดูกล้องวงจรปิดไม่ยอมให้ดูเลยต้องขู่ว่าจะแจ้งความถึงยอมคืนให้ แต่ไม่มีคำขอโทษใดๆ จาก พนง.-ติด…See more
43 สาขาพัทลุง เหม็นคาวมากกกกกกกกก แถมแผนกของกินของสดแม่งไปไหนหมด มีแต่เสื้อผ้าละ
56 ของมึงไม่ต้องถูกมากก็ได้ เน้นคุณภาพบ้าง ไส้กรอกเน่า แต่ยังไม่หมดอายุงี้อ่ะ (สาขาฟอร์จูน) ขนมปังขึ้นราแล้วยังวางขายอีก (สาขาลาดพร้าว) กีวี่หนอนเข้า กะหล่ำปลีเน่าใน ผักบุ้งเน่าแทรกในมัด (สาขาอ่อนนุช)
60 ของป้ายเหลืองพวกผัก เน่าจนไม่รู้จะเน่ายังไง คือทิ้งๆไปก็ได้มั่ง
68 ช่องจ่ายเยอะ เยอะเกินพนง.ใช่ไหม ไ

In [ ]:
w3 = Widget(result, marks, 'tokenized_comment')

In [ ]:
seed = 164
compare = 145

a = numpy.array(onehot_corpus.iloc[seed]).reshape(1, -1)
b = numpy.array(onehot_corpus.iloc[compare]).reshape(1, -1)
print(cosine_similarity(a,b))
print(filtered_corpus[seed])
print(filtered_corpus[compare])

# print(sims[12])